In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.8 MB/s eta 0:00:00


In [2]:
import torch
torch.cuda.empty_cache()

In [9]:
import pandas as pd
import numpy as np
import torch

In [10]:
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from nltk.stem import WordNetLemmatizer

In [11]:
import tensorflow as tf

In [12]:
from transformers.optimization import Adafactor, AdafactorSchedule

In [13]:
from transformers import EarlyStoppingCallback

In [14]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [15]:
data=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ELP/train.csv', error_bad_lines=False)
data.head()

<ipython-input-15-0a952ed0e9f5>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ELP/train.csv', error_bad_lines=False)
Skipping line 3530: expected 8 fields, saw 10
Skipping line 3994: expected 8 fields, saw 9
Skipping line 3996: expected 8 fields, saw 12
Skipping line 4382: expected 8 fields, saw 10



,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [16]:
# Preprocess the data
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
lemmatizer = WordNetLemmatizer()


In [17]:

def preprocess_text(text):
    # lemmatize the text
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    # tokenize the text
    input_ids = tokenizer.encode(text, truncation=True, max_length=512)
    return input_ids

In [18]:
data['input_ids'] = data['full_text'].apply(preprocess_text)

In [19]:
# Fine-tune GPT-2 on your dataset
config = GPT2Config.from_pretrained('gpt2')
config.num_labels = 6
model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)


In [20]:
# Add a padding token to the tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Using pad_token, but it is not set yet.


In [21]:
# Update the model's embeddings to include the new padding token
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [22]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, data, block_size):
        self.tokenizer = tokenizer
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = self.data.iloc[idx]['input_ids']
        labels = self.data.iloc[idx][['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values

        # Pad input_ids
        input_ids = input_ids + [tokenizer.pad_token_id] * (self.block_size - len(input_ids))

        # Convert input_ids and labels to tensors
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        labels = torch.tensor(pd.to_numeric(labels.flatten(), errors='coerce'), dtype=torch.float64)

        return {'input_ids': input_ids, 'labels': labels}

train_data, val_data = train_test_split(data, test_size=0.2)
train_dataset = CustomDataset(tokenizer=tokenizer, data=train_data, block_size=512)
val_dataset = CustomDataset(tokenizer=tokenizer, data=val_data, block_size=512)

In [33]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_steps=50,
    save_steps=50,
    logging_steps=10,
    evaluation_strategy='steps',
    save_total_limit=1,
    gradient_accumulation_steps=4,
    # fp16=True,
    # load_best_model_at_end=True,
    learning_rate=1e-4,
    warmup_steps=100,
    weight_decay=0.01,
   
)

In [31]:
callback = EarlyStoppingCallback(early_stopping_patience=3)

In [27]:
optimizer = Adafactor(
    model.parameters(),
    lr=1e-4,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.01,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)

lr_scheduler = AdafactorSchedule(optimizer)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # optimizers=(optimizer, lr_scheduler),
    # callbacks = [callback],
)

In [35]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: ignored

In [ ]:
trainer.evaluate(val_dataset)

{'eval_loss': 2.9633214473724365,
 'eval_runtime': 30.0601,
 'eval_samples_per_second': 40.386,
 'eval_steps_per_second': 10.113,
 'epoch': 3.0}

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 3            |        cudaMalloc retries: 4         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  13032 MiB |  13897 MiB |  57937 MiB |  44905 MiB |
|       from large pool |  13018 MiB |  13882 MiB |  57821 MiB |  44803 MiB |
|       from small pool |     14 MiB |     15 MiB |    116 MiB |    101 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  13032 MiB |  13897 MiB |  57937 MiB |  44905 MiB |
|       from large pool |  13018 MiB |  13882 MiB |  57821 MiB |

In [32]:
from numba import cuda
device=cuda.get_current_device()
device.reset()